In [1]:
cd ..

/home/ayush/Documents/AI/Projects/Toxic-Comment-Analyser


/home/ayush/Documents/AI/Projects/Toxic-Comment-Analyser/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dataclasses import dataclass

@dataclass
class DataTranformationConfig:
    root_dir: str
    data_yt_path: str
    clean_data_yt_path: str


In [3]:
from src.constants import *
from src.utils.common import read_yaml_file,create_dir

In [4]:
class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,
                 param=PARAM_FILE_PATH):
        self.config=read_yaml_file(config)
        self.param=read_yaml_file(param)
        create_dir([self.config.artifact_root])

    def get_data_transformation(self)->DataTranformationConfig:
        config = self.config.data_transformation
        create_dir([config.root_dir])
        get_dataTran=DataTranformationConfig(
            root_dir=config.root_dir,
            clean_data_yt_path=config.clean_data_yt_path,
            data_yt_path=config.data_yt_path
        )
        return get_dataTran

In [5]:
from src.logging import logger
from text_prettifier import TextPrettifier
from langdetect import detect


In [6]:
class DataTranformation:
    def __init__(self,config=DataTranformationConfig):
        self.config= config
        self.prettifier = TextPrettifier()

        with open(self.config.data_yt_path, "r", encoding="utf-8") as f:
                self.text = [line.strip() for line in f.readlines()]
                logger.info("yt Comment Text successfully loaded")


    def text_cleaner(self,text):
        text = text.lower()
        text = self.prettifier.remove_contractions(text)
        text = self.prettifier.remove_emojis(text)
        text = self.prettifier.remove_html_tags(text)
        text = self.prettifier.remove_urls(text)
        text = self.prettifier.remove_special_chars(text)
        text = self.prettifier.remove_stopwords(text)
        text = self.prettifier.remove_numbers(text)
        return text
    def is_english(self,text):
        try:
            return detect(text) == 'en'
        except:
            return False
        
    def filter_text(self):
        cleaned_text = [self.text_cleaner(t) for t in self.text]
        english_comments = [text for text in cleaned_text if self.is_english(text)]
        with open(self.config.clean_data_yt_path, "w", encoding="utf-8") as f:
            for comment in english_comments:
                f.write(comment + "\n")
            logger.debug('yt Comment Text successfully saved in file')


In [7]:
config = ConfigurationManager()
get_datatranConfig= config.get_data_transformation()
dataingestion = DataTranformation(config=get_datatranConfig)
dataingestion.filter_text()

[2025-06-15 13:31:37,921 : INFO : common : Created directory at: artifacts]
[2025-06-15 13:31:37,923 : INFO : common : Created directory at: artifacts/data_transformation]
[2025-06-15 13:31:37,925 : INFO : 1325234647 : yt Comment Text successfully loaded]
